# AWS Bedrock Tutorial

Amazon Bedrock is a fully managed service that provides access to foundation models from various AI providers through a single API.

In this notebook we'll:
1. Use the **Converse API** - the simplest way to interact with Bedrock models
2. Compare two models on the same prompt:
   - **Amazon Nova Micro** - Amazon's fastest/cheapest text model
   - **Mistral 7B** - Popular open-source model

## Setup

Create a Bedrock client. Credentials are picked up automatically from `~/.aws/credentials`.

In [ ]:
import boto3

bedrock = boto3.client("bedrock-runtime")

## Helper Function

A simple wrapper around the Converse API that also calculates cost.

In [ ]:
# Pricing per 1K tokens (check aws.amazon.com/bedrock/pricing for current rates)
# Note: In EU regions, use inference profile IDs (eu.amazon.nova-*) instead of model IDs
PRICING = {
    "eu.amazon.nova-micro-v1:0": {"input": 0.000035, "output": 0.00014},
    "mistral.mistral-7b-instruct-v0:2": {"input": 0.00015, "output": 0.0002},
}


def ask_bedrock(model_id, prompt, max_tokens=512, temperature=0.7):
    """Send a prompt to Bedrock and return the response."""
    response = bedrock.converse(
        modelId=model_id,
        messages=[{"role": "user", "content": [{"text": prompt}]}],
        inferenceConfig={"maxTokens": max_tokens, "temperature": temperature},
    )

    text = response["output"]["message"]["content"][0]["text"]
    usage = response["usage"]

    # Calculate cost
    price = PRICING[model_id]
    cost = (usage["inputTokens"] * price["input"] + usage["outputTokens"] * price["output"]) / 1000

    print(f"Tokens: {usage['inputTokens']} in, {usage['outputTokens']} out")
    print(f"Cost: ${cost:.6f}")

    return text

## Test with Amazon Nova Micro

In [ ]:
prompt = """Extract the following from this text and return as JSON:
- names (list) - in alphabetical order
- locations (list) - in alphabetical order
- dates (list) - use the same format as Python's datetime.isoformat()

Text: "John met Sarah in Paris on March 15th. They later traveled to London where they met Bob on April 2nd."
"""

In [ ]:
print("=== Amazon Nova Micro ===")
print(ask_bedrock("eu.amazon.nova-micro-v1:0", prompt))

## Test with Mistral 7B

In [ ]:
print("=== Mistral 7B ===")
print(ask_bedrock("mistral.mistral-7b-instruct-v0:2", prompt))

### Add format specification and date as context

In [ ]:
import datetime

prompt = f"""Extract the following from this text and return as JSON:
- names (list in alphabetical order)
- locations (list in alphabetical order)
- dates (list) use the YYYY-MM-DD format

Respond with pretty-printed JSON content only.

Text: "John met Sarah in Paris on March 15th. They later traveled to London where they met Bob on April 2nd."

Context: The current year is {datetime.datetime.now().year}
"""

print(prompt)

In [ ]:
print("=== Amazon Nova Micro ===")
print(ask_bedrock("eu.amazon.nova-micro-v1:0", prompt))

print("=== Mistral 7B ===")
print(ask_bedrock("mistral.mistral-7b-instruct-v0:2", prompt))